In [1]:
import holoocean
import numpy as np
from pynput import keyboard
from typing import Tuple, List, Dict
import time
import my_configs as config
from agent_wrapper import agent
from multi_agent_control_test import create_agent_list

# number of environment ticks between control updates
control_ticks_per_update = 5

# Keyboard listener for exiting simulation
pressed_keys = list()
def on_press(key):
    global pressed_keys
    if hasattr(key, 'char'):
        pressed_keys.append(key.char)
        pressed_keys = list(set(pressed_keys))
def on_release(key):
    global pressed_keys
    if hasattr(key, 'char'):
        pressed_keys.remove(key.char)
listener = keyboard.Listener(
    on_press=on_press,
    on_release=on_release)
listener.start()

cfg = config.get_single_agent_test_cfg()

# create list of 75 evently spaced points, forming a circle of radius 10 around the origin
points_list = []
for i in range(365):
    theta = i * 2 * np.pi / 365
    points_list.append(np.array([10 * np.cos(theta), 10 * np.sin(theta), -5]))

# create a holoocean environment
with holoocean.make(scenario_cfg=cfg) as env:
    # Run simulation
    state = env.tick()
    a = create_agent_list(cfg)
    a[0].update_pose(state['PoseSensor'])
    iteration = 0
    curr_point = 0
    ctrl = np.array([0,0,0,0, 1, -1, -1, 1])
    point_lifetime = 0.011
    while iteration < 50000:
        iteration += 1
        # Check for exit key
        if '`' in pressed_keys:
            break
        if iteration % 500 == 0:
            curr_point += 1
        # draw all points, shaded green if visible and red otherwise
        for p in points_list:
            p_list = [c for c in p]
            if a[0].can_see_point(p):
                env.draw_point(p_list, [0, 255, 0], lifetime=point_lifetime)
            else:
                env.draw_point(p_list, [255, 0, 0], thickness=20, lifetime=point_lifetime)
        a[0].update_pose(state['PoseSensor'])
        # if iteration % control_ticks_per_update == 0:
        #     ctrl = a[0].compute_turn_control(points_list[curr_point])
            # ctrl = a[0].compute_turn_control(-1*points_list[0])
        env.act('auv0', ctrl)
        state = env.tick()


/home/jrbeyer/mae577/577-Final-Project/agent_wrapper.py:197: RuntimeWarning: invalid value encountered in arccos
  relative_angle = abs(np.arccos(np.dot(d, f)
